In [1]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras
from inception_blocks_v2 import * 
from fr_utils import *
import keras.backend as k
k.set_image_data_format("channels_first")

In [2]:
encoder = faceRecoModel((3,96,96))

In [3]:
encoder.summary()

Model: "FaceRecoModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 3, 96, 96)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding2d                │ (None, 3, 102, 102)       │               0 │ input_layer[0][0]          │
│ (ZeroPadding2D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1 (Conv2D)                │ (None, 64, 48, 48)        │           9,472 │ zero_padding2d[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn1 (BatchNormalization)      │ (None, 64, 48, 48)        │             256 │ conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 64, 48, 48)        │               0 │ bn1[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding2d_1              │ (None, 64, 50, 50)        │               0 │ activation[0][0]           │
│ (ZeroPadding2D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 64, 24, 24)        │               0 │ zero_padding2d_1[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2 (Conv2D)                │ (None, 64, 24, 24)        │           4,160 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn2 (BatchNormalization)      │ (None, 64, 24, 24)        │             256 │ conv2[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 64, 24, 24)        │               0 │ bn2[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding2d_2              │ (None, 64, 26, 26)        │               0 │ activation_1[0][0]         │
│ (ZeroPadding2D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3 (Conv2D)                │ (None, 192, 24, 24)       │         110,784 │ zero_padding2d_2[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn3 (BatchNormalization)      │ (None, 192, 24, 24)       │             768 │ conv3[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 192, 24, 24)       │               0 │ bn3[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding2d_3              │ (None, 192, 26, 26)       │               0 │ activation_2[0][0]         │
│ (ZeroPadding2D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 3,743,280 (14.28 MB)

 Trainable params: 3,733,968 (14.24 MB)

 Non-trainable params: 9,312 (36.38 KB)

In [8]:
def triplet_loss(y_true,y_pred,alpha=0.2):
    """
    this is a loss for face recognision and saiumi network 

    arguments :
    t_true -- we dont use it , this is just for keras to accept our function as loss
    y_pred -- python list containing 3 object :
        anchor -- the main image of a person 
        positive -- another image of anchor person face 
        negative -- a image dosent belong to anchor person 

    return :
    loss -- calculated loss 
    """
    anchor , positive , negative = y_pred 
    distance_positive = tf.reduce_sum(tf.square(anchor-positive),axis=-1)
    distance_negative = tf.reduce_sum(tf.square(anchor-negative),axis=-1)
    temp = distance_positive+alpha-distance_negative
    loss = tf.reduce_sum(tf.maximum(temp, 0.0))
    return loss

In [9]:
with tf.compat.v1.Session() as test:
    tf.compat.v1.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random.normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random.normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random.normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

loss = 528.1426


**Expected Output**:

<table>
    <tr>
        <td>
            **loss**
        </td>
        <td>
           528.143
        </td>
    </tr>

</table>

In [12]:
encoder.compile(
    optimizer = "adam",
    loss = triplet_loss,
    metrics = ["accuracy"]   
)

In [15]:
load_weights_from_FaceNet(encoder)

In [16]:
database = {}
FRmodel = encoder
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

(1, 3, 96, 96)
(1, 3, 96, 96)
(1, 3, 96, 96)
(1, 3, 96, 96)
(1, 3, 96, 96)
(1, 3, 96, 96)
(1, 3, 96, 96)
(1, 3, 96, 96)
(1, 3, 96, 96)
(1, 3, 96, 96)
(1, 3, 96, 96)
(1, 3, 96, 96)


In [19]:
#images\arastoo.jpg"
database["arastoo"] = img_to_encoding("images/arastoo.jpg",FRmodel)

(1, 3, 96, 96)


In [33]:
def verify(image_path , id , database , model):
    """
    this will implement verification for a image and expacted id 

    arguments:
    image_path -- path of a input image
    id -- expaected id 
    database -- dctinary including encoding 
    model -- saiumi net for encoding the input image 

    returns :
    distance -- distance between image and id 
    valid -- boolean for accepting the image 
    """
    id_vector = database[id]
    image_vector = img_to_encoding(image_path, model)
    distance = tf.reduce_sum(tf.square(id_vector-image_vector))
    valid = distance < 0.7 

    return tf.math.sqrt(distance).numpy(), valid.numpy()

In [34]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

(1, 3, 96, 96)


(0.6593838, True)

In [35]:
verify("images/aras_1.jpg", "younes", database, FRmodel)

(1, 3, 96, 96)


(1.0059596, False)

In [36]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

(1, 3, 96, 96)


(0.86225706, False)

In [65]:
def face_recognization(image_path , database , model):
    """
    this will implement verification for a image and expacted id 

    arguments:
    image_path -- path of a input image
    database -- dctinary including encoding 
    model -- saiumi net for encoding the input image 

    returns :
    distance -- distance between image and id 
    id -- name of person 
    """
    vector_input = img_to_encoding(image_path,model)
    distance =10000
    id = ""
    for name , encode in database.items():
        
        diff = np.linalg.norm(encode-vector_input)
        if diff < distance :
            distance = diff
            id = name 
            
    
   

    return distance , id 

In [66]:
face_recognization("images/camera_0.jpg", database, FRmodel)

(1, 3, 96, 96)


(0.6593838, 'younes')

In [67]:
who_is_it("images/aras_1.jpg", database, FRmodel)

(1, 3, 96, 96)
Not in the database.


(0.7979062, 'tian')

In [69]:
who_is_it("images/aras_2.png", database, FRmodel)

(1, 3, 96, 96)
Not in the database.


(0.7672411, 'danielle')